# 노트 정확도

- Note on, Note off 리스트를 만들어 On 시간에 따른 On, off 만들기

- 입력데이터와 타깃데이터는 모두 csv 정형 데이터, 엑설 파일

- mido 라이브러리는 note 값으로 0~127까지의 값을 갖는다.

In [1]:
import pandas as pd

def load_csv_files(input_file_name, target_file_name):
    """
    input_file_name  = 훈련자 데이터
    target_file_name = 정답 데이터
    """
    input_file = pd.read_csv(input_file_name)
    target_file = pd.read_csv(target_file_name)
    
    return input_file, target_file

In [2]:
input_file, target_file = load_csv_files('./midi_data/train_data0307.csv', './midi_data/target_data0307.csv')

# 위 아래 행간 비교를 통해 음정 정확도를 측정하기

- 임계치 만큼의 피아노 상태를 추적한다.
- 인풋 데이터의 한 행에 대하여, 타깃 데이터의 앞 뒤로 x행 만큼의 상태를 비교해서, 맞는 행이 있다면 음정을 정확히 친 것으로 간주한다.
- 임계값에 따라 난이도를 측정할 수 있다.

# 아래 코드는 개발 했던 코드 보호 차원에서 남겨둠

In [72]:
target_temp_data = []

target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
input_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]


total_score = 0
input_data_len = len(input_file)


for index in range(len(input_file)):
        
        # 초기화
        target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
        input_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
        target_temp_data = []
    
        notes_input = eval(input_file.at[index, 'note']) # 문자열을 -> 숫자로
        msg_type_input = eval(input_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
        
        # 임계값을 기준으로 앞, 뒤 행 가져오기
        for i in range(1+1):
            if i == 0:
                notes_target = eval(target_file.at[index, 'note']) # 문자열을 -> 숫자로
                msg_type_target = eval(target_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
                target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정
            else:
                try:
                    notes_target = eval(target_file.at[index-i, 'note']) # 문자열을 -> 숫자로
                    msg_type_target = eval(target_file.at[index-i, 'msg_type']) # 문자열을 -> 숫자로
                    target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정
                except:
                    pass
                notes_target = eval(target_file.at[index+i, 'note']) # 문자열을 -> 숫자로
                msg_type_target = eval(target_file.at[index+i, 'msg_type']) # 문자열을 -> 숫자로
                target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

        temp = list(zip(notes_input, msg_type_input)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정
        
        
        # 피아노 활성화 값 추적 
        for data in temp:
            if data[1] == 'note_on':
                input_file_note_status[data[0]][1]['status'] = True
            elif data[1] == 'note_off':
                input_file_note_status[data[0]][1]['status'] = False
        
        print(index,'번쨰', temp, '검사 시작')
        # 이 곳에서 비교 알고리즘을 작성한다. 
        for target_data in target_temp_data:
            print('='*50)
            print('앞뒤행: ',target_data)
            target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)] # 타깃 데이터의 피아노 상태 초기화.
            print(temp,'에 대한', target_data, '평가')
            for data in target_data:
                if data[1] == 'note_on':
                    target_file_note_status[data[0]][1]['status'] = True
                elif data[1] == 'note_off':
                    target_file_note_status[data[0]][1]['status'] = False
            # 위 코드로 타깃 피아노 상태 만든다.
            
            
            
            
            # 전체 피아노 상태가 같아야 같은 것으로 간주할 수 있음. 다른게 있으면 break?
            isPerfect = True
            
            for i in range(len(input_file_note_status)):
                if input_file_note_status[i][1]['status'] != target_file_note_status[i][1]['status']:
                    print('다른 노트 발견..', input_file_note_status[i][0]['note'], target_file_note_status[i][0]['note'])
                    isPerfect = False
                    print('='*50)
                    break
                # else에 도달 했다는 것은 인풋과 타깃의 피아노 상태가 일치하다는 것을 의미.
                # 스코어 ++
            
            if isPerfect:
                print('일치! 점수 추가!')
                print('='*50)
                total_score += 1
                break
                    
        
            
        print('==' * 50)

print('임계값 1에 따른 음정 정확도 결과: ', int(total_score / input_data_len * 100), '%' )

0 번쨰 [] 검사 시작
앞뒤행:  [(60, 'note_on')]
[] 에 대한 [(60, 'note_on')] 평가
다른 노트 발견.. 60 60
앞뒤행:  []
[] 에 대한 [] 평가
일치! 점수 추가!
1 번쨰 [(60, 'note_on'), (60, 'note_on')] 검사 시작
앞뒤행:  []
[(60, 'note_on'), (60, 'note_on')] 에 대한 [] 평가
다른 노트 발견.. 60 60
앞뒤행:  [(60, 'note_on')]
[(60, 'note_on'), (60, 'note_on')] 에 대한 [(60, 'note_on')] 평가
일치! 점수 추가!
2 번쨰 [] 검사 시작
앞뒤행:  [(64, 'note_on')]
[] 에 대한 [(64, 'note_on')] 평가
다른 노트 발견.. 64 64
앞뒤행:  []
[] 에 대한 [] 평가
일치! 점수 추가!
3 번쨰 [(64, 'note_on'), (64, 'note_on')] 검사 시작
앞뒤행:  []
[(64, 'note_on'), (64, 'note_on')] 에 대한 [] 평가
다른 노트 발견.. 64 64
앞뒤행:  [(64, 'note_on')]
[(64, 'note_on'), (64, 'note_on')] 에 대한 [(64, 'note_on')] 평가
일치! 점수 추가!
4 번쨰 [] 검사 시작
앞뒤행:  [(67, 'note_on')]
[] 에 대한 [(67, 'note_on')] 평가
다른 노트 발견.. 67 67
앞뒤행:  []
[] 에 대한 [] 평가
일치! 점수 추가!
5 번쨰 [(67, 'note_on'), (67, 'note_on')] 검사 시작
앞뒤행:  []
[(67, 'note_on'), (67, 'note_on')] 에 대한 [] 평가
다른 노트 발견.. 67 67
앞뒤행:  [(67, 'note_on')]
[(67, 'note_on'), (67, 'note_on')] 에 대한 [(67, 'note_on')] 평가
일치! 점수 추가!
6 번쨰

949 번쨰 [(60, 'note_on'), (60, 'note_on'), (57, 'note_off'), (57, 'note_off')] 검사 시작
앞뒤행:  [(66, 'note_off')]
[(60, 'note_on'), (60, 'note_on'), (57, 'note_off'), (57, 'note_off')] 에 대한 [(66, 'note_off')] 평가
다른 노트 발견.. 60 60
앞뒤행:  []
[(60, 'note_on'), (60, 'note_on'), (57, 'note_off'), (57, 'note_off')] 에 대한 [] 평가
다른 노트 발견.. 60 60
앞뒤행:  [(57, 'note_on')]
[(60, 'note_on'), (60, 'note_on'), (57, 'note_off'), (57, 'note_off')] 에 대한 [(57, 'note_on')] 평가
다른 노트 발견.. 57 57
950 번쨰 [(43, 'note_off'), (43, 'note_off')] 검사 시작
앞뒤행:  [(57, 'note_on')]
[(43, 'note_off'), (43, 'note_off')] 에 대한 [(57, 'note_on')] 평가
다른 노트 발견.. 57 57
앞뒤행:  [(66, 'note_off')]
[(43, 'note_off'), (43, 'note_off')] 에 대한 [(66, 'note_off')] 평가
일치! 점수 추가!
951 번쨰 [(51, 'note_off'), (51, 'note_off'), (66, 'note_on'), (66, 'note_on'), (60, 'note_off'), (60, 'note_off')] 검사 시작
앞뒤행:  []
[(51, 'note_off'), (51, 'note_off'), (66, 'note_on'), (66, 'note_on'), (60, 'note_off'), (60, 'note_off')] 에 대한 [] 평가
다른 노트 발견.. 66 66
앞뒤행:  [(57, 

# 음정 정확도 함수

In [78]:

def evaluate_note_accuracy(input_file, target_file, threshold=1):
    """
    threshold = 행 간 앞뒤로 몇 칸씩 조사할지? 정하는 매게변수
    input_file = 인풋 파일 Dataframe으로 불러올 것
    target_file = 타깃 파일 Dataframe으로 불러올 것
    """
    
    
    target_temp_data = [] # 타깃 데이터의 앞 뒤 행을 임시로 저장하기 위한 변수
    # 아래는 피아노 Note On의 상태를 조사하기 위한 리스트
    target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
    input_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]

    
    total_score = 0 # 총점을 측정하기 위해서
    input_data_len = len(input_file) # 총점을 인풋 데이터의 행의 개수로 나눠 맞은 행의 개수 비율을 측정하기 위해 필요.


    for index in range(len(input_file)):

            # 초기화
            target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
            input_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)]
            target_temp_data = []

            notes_input = eval(input_file.at[index, 'note']) # 문자열을 -> 숫자로
            msg_type_input = eval(input_file.at[index, 'msg_type']) # 문자열을 -> 숫자로

            # 임계값을 기준으로 앞, 뒤 행 가져오기
            for i in range(1+threshold):
                if i == 0:
                    notes_target = eval(target_file.at[index, 'note']) # 문자열을 -> 숫자로
                    msg_type_target = eval(target_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
                    target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정
                else:
                    try:
                        notes_target = eval(target_file.at[index-i, 'note']) # 문자열을 -> 숫자로
                        msg_type_target = eval(target_file.at[index-i, 'msg_type']) # 문자열을 -> 숫자로
                        target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정
                    except:
                        pass
                    notes_target = eval(target_file.at[index+i, 'note']) # 문자열을 -> 숫자로
                    msg_type_target = eval(target_file.at[index+i, 'msg_type']) # 문자열을 -> 숫자로
                    target_temp_data.append(list(zip(notes_target, msg_type_target)))# [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

            temp = list(zip(notes_input, msg_type_input)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정


            # 피아노 활성화 값 추적 
            for data in temp:
                if data[1] == 'note_on':
                    input_file_note_status[data[0]][1]['status'] = True
                elif data[1] == 'note_off':
                    input_file_note_status[data[0]][1]['status'] = False

            # 이 곳에서 비교 알고리즘을 작성한다. 
            for target_data in target_temp_data:
                target_file_note_status = [[{'note': x}, {'status': False}] for x in range(128)] # 타깃 데이터의 피아노 상태 초기화.
                for data in target_data:
                    if data[1] == 'note_on':
                        target_file_note_status[data[0]][1]['status'] = True
                    elif data[1] == 'note_off':
                        target_file_note_status[data[0]][1]['status'] = False
                # 위 코드로 타깃 피아노 상태 만든다.


                # 전체 피아노 상태가 같아야 같은 것으로 간주할 수 있음. 다른게 있으면 break?
                isPerfect = True

                for i in range(len(input_file_note_status)):
                    if input_file_note_status[i][1]['status'] != target_file_note_status[i][1]['status']:
                        isPerfect = False
                        break
                    # else에 도달 했다는 것은 인풋과 타깃의 피아노 상태가 일치하다는 것을 의미.
                    # 스코어 ++

                if isPerfect:
                    total_score += 1
                    break

    print('임계값 1에 따른 음정 정확도 결과: ', int(total_score / input_data_len * 100), '%' )

In [79]:
evaluate_note_accuracy(input_file, target_file)

임계값 1에 따른 음정 정확도 결과:  91 %


# 음정 정확도 함수 조회

In [80]:
evaluate_note_accuracy?

# 음정 정확도 함수 사용 예시

In [81]:
import pandas as pd

def load_csv_files(input_file_name, target_file_name):
    """
    input_file_name  = 훈련자 데이터
    target_file_name = 정답 데이터
    """
    input_file = pd.read_csv(input_file_name)
    target_file = pd.read_csv(target_file_name)
    
    return input_file, target_file

input_file, target_file = load_csv_files('./midi_data/train_data0307.csv', './midi_data/target_data0307.csv')

evaluate_note_accuracy(input_file, target_file)

임계값 1에 따른 음정 정확도 결과:  91 %
